In [157]:
# Imports

import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import json

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

# BERT model, tokenizer

from transformers import TFBertForSequenceClassification, TFAutoModel , AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")

model = TFBertForSequenceClassification.from_pretrained("SZTAKI-HLT/hubert-base-cc", num_labels=3, output_attentions=False, output_hidden_states=False)

# Constants

SHUFFLE_RANDOM_STATE = 42
TRAIN_RANDOM_STATE = 42

TEXT = 'Sentence'
FILTER = 'Entity'
START_TOKEN = 'START'
TOKEN_LEN = 'LEN'
Y_HEADER = 'LABEL'
LABELS = {
    "NEG": 0,
    "SEM": 1,
    "POZ": 2
}

EKEZET_VALTAS_RAGNAL = {
    'a': 'á',
    'e': 'é'
}

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at SZTAKI-HLT/hubert-base-cc and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [158]:
# Text processing

fix = 0

def extend_context(text, context, left_index, right_index):
    # TODO: Mi van, ha a neveket sem vesszuk bele?
    global fix
    if type(text) != list:
        text = text.split()
    start_size = 0
    while start_size < 10:
        if left_index > 0:
            left_index -= 1
            context.insert(0, text[left_index])
            if len(text[left_index]) > 1:
                start_size += 1
        if right_index < len(text) -1:
            right_index += 1
            context.append(text[right_index])
            if len(text[right_index]) > 1:
                start_size += 1
        if left_index <= 0 and right_index >= len(text) -1:
            # fix += 1
            break
    return ' '.join(context)

def contextualize(index, debug = False):
    global fix
    text = ' '.join([i for i in re.split(r'( - |(?![%.-])\W)|(-e[\n ])', dataset[TEXT][index]) if i])
    context_start_index = int(dataset[START_TOKEN][index] - 1)
    context_stop_index = int(context_start_index+dataset[TOKEN_LEN][index] - 1)

    context_name_tokens = \
        [t for t in text
                        .split()[context_start_index:context_stop_index+1]]

    context_name = ' '.join(context_name_tokens)
    context_filter = dataset[FILTER][index]

    # Ha ragozunk, akkor a-bol á lesz pl Csaba -> Csabával
    if len(context_name) >= len(context_filter) \
            and context_filter[len(context_filter)-1] in EKEZET_VALTAS_RAGNAL\
            and EKEZET_VALTAS_RAGNAL.get(context_filter[len(context_filter)-1]) == context_name[len(context_filter)-1]:
        context_filter = context_filter[:-1] + context_name[len(context_filter)-1]

    context_list = context_name_tokens
    left_index = context_start_index
    right_index = context_stop_index
    context = extend_context(text, context_list, left_index, right_index)
    return context

    # if len(context_name) < len(filter):
    #     if debug:
    #         print("rovidebb")
    #         print(dataset['ID'][index])
    #         print(dataset[START_TOKEN][index])
    #         print(dataset[FILTER][index])
    #         print(text)
    #         print(text.split())
    #         print(context_name_tokens)
    #         print('-----')
    #     fix += 1
    # elif not context_name.startswith(filter)\
    #         and filter[len(filter)-1] in EKEZET_VALTAS_RAGNAL\
    #         and EKEZET_VALTAS_RAGNAL.get(filter[len(filter)-1]) != context_name[len(filter)-1]:
    #
    #     if debug:
    #         print(EKEZET_VALTAS_RAGNAL.get(filter[len(filter)-1]))
    #         print(context_name)
    #         print(EKEZET_VALTAS_RAGNAL.get(filter[len(filter)-1]) == context_name[len(filter)-1])
    #         print("nem ilyen izes")
    #         print(dataset['ID'][index])
    #         print(dataset[START_TOKEN][index])
    #         print(dataset[FILTER][index])
    #         print(text)
    #         print(text.split())
    #         print(context_name_tokens)
    #         print('-----')
    #     fix += 1

In [159]:
# Load and set up data

dataset = pd.read_csv('db/train.csv', sep=';')
dataset[Y_HEADER] = dataset[Y_HEADER].map(LABELS)
dataset = shuffle(dataset, random_state=SHUFFLE_RANDOM_STATE)
dataset.info()

X_list = []
print(len(dataset.index))
for i  in range(len(dataset.index)):
    X_list.append(contextualize(i))

# print ('need fix: ', fix)

X = np.asarray(X_list)
y = dataset[Y_HEADER].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=TRAIN_RANDOM_STATE)

print('X_train[0]: ', X_train[0])
print('y_train[0]: ', y_train[0])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7700 entries, 3856 to 7270
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        7700 non-null   int64 
 1   START     7700 non-null   int64 
 2   LEN       7700 non-null   int64 
 3   Entity    7700 non-null   object
 4   Sentence  7700 non-null   object
 5   URL       7700 non-null   object
 6   LABEL     7700 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 481.2+ KB
7700
X_train[0]:  A miskolci diákolimpiai fináléban Fejér Barna és Cs. Németh Péter fiatal tanítványai kilenc
y_train[0]:  1


In [160]:
# Peek tokenizer
print('Vocab size:', len(tokenizer.vocab))

Vocab size: 32001


In [161]:
# Tokenize a sentance

tokens = tokenizer.tokenize("Szia Világ!")
print('tokens:',tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print('ids:', ids)

cls_token, sep_token = tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]'])
print('cls token:', cls_token, 'sep token:', sep_token)

tokens: ['Szia', 'Világ', '!']
ids: [15689, 6976, 8189]
cls token: 2 sep token: 3


In [162]:
def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(X):
    sentence = tf.ragged.constant([
        encode_sentence(s) for s in X])

    cls = [[cls_token]]*sentence.shape[0]
    input_word_ids = tf.concat([cls, sentence], axis=-1)

    input_mask = tf.ones_like(input_word_ids).to_tensor()

    type_cls = tf.zeros_like(cls)
    type_s = tf.ones_like(sentence)
    input_type_ids = tf.concat([type_cls, type_s], axis=-1).to_tensor()

    inputs = {
      'input_word_ids': input_word_ids.to_tensor(),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

    return inputs

In [163]:
bert_train = bert_encode(X_train)
bert_test = bert_encode(X_test)

In [164]:
for key, value in bert_train.items():
  print(f'{key:18s} shape: {value.shape}')

print(f'\ntrain_labels {"":5s} shape: {y_train.shape}')

input_word_ids     shape: (6160, 46)
input_mask         shape: (6160, 46)
input_type_ids     shape: (6160, 46)

train_labels       shape: (6160,)


In [165]:
# Ha a model TFBertForSequenceClassification

print(model.config)

# config_dict = json.loads(tf.io.gfile.GFile('bertconfig.json').read())
#
# bert_config = bert.configs.BertConfig.from_dict(config_dict)
#
# config_dict

BertConfig {
  "_name_or_path": "SZTAKI-HLT/hubert-base-cc",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32001
}



In [166]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=48)

In [167]:
X_batch = {key: val[:10] for key, val in bert_train.items()}
for i in range(10):
    print(y_train[i], ':', X_train[i], '\n')

model(
    X_batch['input_word_ids']
)

1 : A miskolci diákolimpiai fináléban Fejér Barna és Cs. Németh Péter fiatal tanítványai kilenc 

1 : Hétfőn tett tanuvallomást gyanúsítottként Gyurcsány Ferenc , az erről készült kb. fél órás 

0 : A kormányoldal részéről mindössze az előterjesztő ( Lázár János ) , a vezérszónok ( Szijjártó Péter ) és két 

1 : Gáspár Győző felesége , Bea asszony számára legfontosabb a család , viszi a háztartást és szerinte 

1 : A szerződést Vecsei Miklós , a Magyar Máltai Szeretetszolgálat alelnöke és Aknai Zoltán , a Menhely Alapítvány 

1 : ki szerdán a Kulturális Örökségvédelmi Hivatal ( KÖH ) elnökévé Réthelyi Miklós nemzeti erőforrás miniszter. 

1 : Ismeretlenek betörtek Tőkés László budapesti , VI. kerületi irodájába. 

1 : a kormány - mondta az InfoRádiónak Bod Péter Ákos az Adózás Európában elnevezésű nemzetközi konferencián. 

1 : és az éppen csak a napi étkezésre elég - közölte a kormányzó szóvivője , Rudi Iriawan. 

0 : Javier Morena egy szegény család legidősebb gyermeke volt , gyümölc

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[ 0.32845503,  0.07863181, -0.20424253],
       [ 0.08015388, -0.44066608, -0.18912083],
       [ 0.21136162, -0.26187128, -0.01093509],
       [-0.15032732, -0.284038  ,  0.00367391],
       [ 0.01367137, -0.30101752,  0.07181562],
       [ 0.11750799, -0.55502206, -0.02448407],
       [ 0.02691292, -0.36814243,  0.01793264],
       [ 0.2017272 , -0.3457285 , -0.0309227 ],
       [ 0.10670129, -0.5114079 , -0.41897744],
       [-0.16983794, -0.17508838, -0.05690034]], dtype=float32)>, hidden_states=None, attentions=None)

In [168]:
print(model.summary())
for i, layer in enumerate (model.layers):
    print (i, layer)
    try:
        print ("    ",layer.activation)
    except AttributeError:
        print('   no activation attribute')

Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  110618112 
_________________________________________________________________
dropout_413 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 110,620,419
Trainable params: 110,620,419
Non-trainable params: 0
_________________________________________________________________
None
0 <transformers.models.bert.modeling_tf_bert.TFBertMainLayer object at 0x000002230CEA5EB0>
   no activation attribute
1 <keras.layers.core.Dropout object at 0x000002230A6E4EB0>
   no activation attribute
2 <keras.layers.core.Dense object at 0x00000222BB57DCA0>
     <function linear at 0x00000222AFE9BAF0>


In [169]:
# Ha a model TFAutoModel

model = TFAutoModel.from_pretrained("SZTAKI-HLT/hubert-base-cc")

print(model.config)

# config_dict = json.loads(tf.io.gfile.GFile('bertconfig.json').read())
#
# bert_config = bert.configs.BertConfig.from_dict(config_dict)
#
# config_dict

Some layers from the model checkpoint at SZTAKI-HLT/hubert-base-cc were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at SZTAKI-HLT/hubert-base-cc.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


BertConfig {
  "_name_or_path": "SZTAKI-HLT/hubert-base-cc",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32001
}



In [170]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=48)

In [171]:
X_batch = {key: val[:10] for key, val in bert_train.items()}
for i in range(10):
    print(y_train[i], ':', X_train[i], '\n')

model(
    X_batch['input_word_ids']
)

1 : A miskolci diákolimpiai fináléban Fejér Barna és Cs. Németh Péter fiatal tanítványai kilenc 

1 : Hétfőn tett tanuvallomást gyanúsítottként Gyurcsány Ferenc , az erről készült kb. fél órás 

0 : A kormányoldal részéről mindössze az előterjesztő ( Lázár János ) , a vezérszónok ( Szijjártó Péter ) és két 

1 : Gáspár Győző felesége , Bea asszony számára legfontosabb a család , viszi a háztartást és szerinte 

1 : A szerződést Vecsei Miklós , a Magyar Máltai Szeretetszolgálat alelnöke és Aknai Zoltán , a Menhely Alapítvány 

1 : ki szerdán a Kulturális Örökségvédelmi Hivatal ( KÖH ) elnökévé Réthelyi Miklós nemzeti erőforrás miniszter. 

1 : Ismeretlenek betörtek Tőkés László budapesti , VI. kerületi irodájába. 

1 : a kormány - mondta az InfoRádiónak Bod Péter Ákos az Adózás Európában elnevezésű nemzetközi konferencián. 

1 : és az éppen csak a napi étkezésre elég - közölte a kormányzó szóvivője , Rudi Iriawan. 

0 : Javier Morena egy szegény család legidősebb gyermeke volt , gyümölc

TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor: shape=(10, 46, 768), dtype=float32, numpy=
array([[[ 8.53495657e-01, -1.59622639e-01,  7.50690877e-01, ...,
         -8.64670396e-01, -3.87077928e-01,  1.39135730e+00],
        [-2.21885860e-01,  3.02363753e-01, -4.67386395e-01, ...,
          9.62088257e-02, -1.54633909e-01,  6.96720183e-03],
        [-1.26337215e-01, -5.24650931e-01, -4.10777330e-01, ...,
         -4.55705345e-01,  1.59798861e-01,  4.36524093e-01],
        ...,
        [ 2.92310238e-01,  4.05387521e-01,  4.38325763e-01, ...,
         -2.59235084e-01, -1.19538799e-01,  1.40259922e-01],
        [ 3.49868774e-01,  3.38483542e-01,  3.77705902e-01, ...,
         -2.63183296e-01, -6.03891760e-02,  1.37556344e-01],
        [ 4.29671288e-01,  2.97923118e-01,  3.64649624e-01, ...,
         -2.38864273e-01, -3.28157097e-02,  1.32595718e-01]],

       [[ 1.31340015e+00, -3.32418203e-01,  5.54044902e-01, ...,
         -6.51495636e-01, -1.00764446e-02,  6.66303694e-01],
 

In [172]:
print(model.summary())
for i, layer in enumerate (model.layers):
    print (i, layer)
    try:
        print ("    ",layer.activation)
    except AttributeError:
        print('   no activation attribute')

Model: "tf_bert_model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  110618112 
Total params: 110,618,112
Trainable params: 110,618,112
Non-trainable params: 0
_________________________________________________________________
None
0 <transformers.models.bert.modeling_tf_bert.TFBertMainLayer object at 0x0000022311E7C730>
   no activation attribute
